In [3]:
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
text_list1 = ["Cash and cash equivalents and current Marketable securities, and $378 million in Prepaid expenses and other current assets.", "The company has recieved $417 million for the payment of interest. "]
text_list3 = ['This loan debt is increasing my financial burden.', 'This loan debt is decreasing my financial burden.']
text_list4 = ['Cash and cash equivalents and current Marketable securities, and $378 million in Prepaid expenses and other current assets.', 
    'Nick has high warranty liability, but he has does not high wealth.', 'Nick has low warranty liability.', 'Nick dont have warranty liability.', 'Nick has warranty liability.','She does not like Steve Jobs but likes Apple products.',
    'He does not like Adolf Hitler but likes German products.', 'There is no English language option.', 'I had very high short term loans.']


In [4]:
from app.common.MySQLUtility import MySQLUtility
import os 

domains = ['esg', 'liabilities' ] #'liabilities', 'esg'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './store/genuine-wording-key.json'

DB_HOST = '34.170.168.203'
DB_USER = 'root'
DB_PASSWORD = 'nu123456'
DB_NAME = 'lca_db'

dbutil = MySQLUtility(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)


In [5]:
from app.TextRank_Extractor import TextRank_Extractor
from app.Risk_Score_Service import Risk_Score_Service
from app.PreProcessText import PreProcessText

rr = Risk_Score_Service(dbutil)
pp = PreProcessText()
keywords = ['loan debt', 'liability', 'short term loan', 'expense', 'asset']

keywords = rr.get_keywords('esg')
key_polarity = []

for kwrd in keywords: 
    key_scr = {}
    key_scr['name'] = str(kwrd)
    key_scr['score'] = 0
    key_scr['count'] = 0
    key_scr['polarity'] = 0
    key_polarity.append(key_scr)

for text in text_list4:
    text_lem = pp.get_lemmantizer(text)
    for key_dic in key_polarity:
        if key_dic['name'] in text_lem:
                key_dic['score'] += rr.get_sentiment_score(text)
                key_dic['count'] += 1

for key_dic in key_polarity:
    if not key_dic['count'] == 0:
        key_dic['polarity'] = (key_dic['score']) / key_dic['count']

print (key_polarity)



[{'name': 'loan debt', 'score': 0, 'count': 0, 'polarity': 0}, {'name': 'liability', 'score': -384.1829240322113, 'count': 4, 'polarity': -96.04573100805283}, {'name': 'short term loan', 'score': -98.54609370231628, 'count': 1, 'polarity': -98.54609370231628}, {'name': 'expense', 'score': -86.26590371131897, 'count': 1, 'polarity': -86.26590371131897}, {'name': 'asset', 'score': -86.26590371131897, 'count': 1, 'polarity': -86.26590371131897}]


In [8]:
import spacy

nlp = spacy.load('en_core_web_md')

for text in text_list4:    
    text_lem = pp.get_lemmantizer(text)
    context_score = 0
    sem_score = rr.get_sentiment_score(text)
    for key_dic in key_polarity:
        if key_dic['name'] in text_lem:            
            pol_score = key_dic['polarity']
            if sem_score < 0:
                context_score = -key_dic['polarity']
            else: 
                context_score = key_dic['polarity']
        else: 
            context_score = rr.get_sentiment_score(text)
    print (text)
    print (context_score)


Cash and cash equivalents and current Marketable securities, and $378 million in Prepaid expenses and other current assets.
86.26590371131897
Nick has high warranty liability, but he has does not high wealth.
-99.78210926055908
Nick has low warranty liability.
-99.80202913284302
Nick dont have warranty liability.
-84.78177189826965
Nick has warranty liability.
-99.81701374053955
She does not like Steve Jobs but likes Apple products.
98.87738227844238
He does not like Adolf Hitler but likes German products.
99.23375248908997
There is no English language option.
-99.95936751365662
I had very high short term loans.
-98.54609370231628


In [7]:
for text in text_list4:  
    print (text) 
    print (rr.get_semantic_score(text)) 
    

Cash and cash equivalents and current Marketable securities, and $378 million in Prepaid expenses and other current assets.
100.0
Nick has high warranty liability, but he has does not high wealth.
100.0
Nick has low warranty liability.
100.0
Nick dont have warranty liability.
100.0
Nick has warranty liability.
100.0
She does not like Steve Jobs but likes Apple products.
100.0
He does not like Adolf Hitler but likes German products.
100.0
There is no English language option.
100.0
I had very high short term loans.
100.0
